In [102]:
import pandas as pd

In [103]:
course_metadata = pd.read_csv(r"csv_dataframe - Sheet1.csv") 

In [104]:
# Create a dictionary where the keys are column names 'lh0' to 'lh14'
columns = [f'lh{i}' for i in range(15)]

# Create a DataFrame with 5 rows and 15 columns
data = [[f'Row{row+1}_Col{col}' for col in range(15)] for row in range(5)]

# Create the DataFrame
timetable = pd.DataFrame(data, columns=columns)

timetable

,lh0,lh1,lh2,lh3,lh4,lh5,lh6,lh7,lh8,lh9,lh10,lh11,lh12,lh13,lh14
0,Row1_Col0,Row1_Col1,Row1_Col2,Row1_Col3,Row1_Col4,Row1_Col5,Row1_Col6,Row1_Col7,Row1_Col8,Row1_Col9,Row1_Col10,Row1_Col11,Row1_Col12,Row1_Col13,Row1_Col14
1,Row2_Col0,Row2_Col1,Row2_Col2,Row2_Col3,Row2_Col4,Row2_Col5,Row2_Col6,Row2_Col7,Row2_Col8,Row2_Col9,Row2_Col10,Row2_Col11,Row2_Col12,Row2_Col13,Row2_Col14
2,Row3_Col0,Row3_Col1,Row3_Col2,Row3_Col3,Row3_Col4,Row3_Col5,Row3_Col6,Row3_Col7,Row3_Col8,Row3_Col9,Row3_Col10,Row3_Col11,Row3_Col12,Row3_Col13,Row3_Col14
3,Row4_Col0,Row4_Col1,Row4_Col2,Row4_Col3,Row4_Col4,Row4_Col5,Row4_Col6,Row4_Col7,Row4_Col8,Row4_Col9,Row4_Col10,Row4_Col11,Row4_Col12,Row4_Col13,Row4_Col14
4,Row5_Col0,Row5_Col1,Row5_Col2,Row5_Col3,Row5_Col4,Row5_Col5,Row5_Col6,Row5_Col7,Row5_Col8,Row5_Col9,Row5_Col10,Row5_Col11,Row5_Col12,Row5_Col13,Row5_Col14


In [105]:
len(course_metadata)

103

In [106]:
import pandas as pd
import random
import numpy as np
from collections import Counter

# Initialize timetable with 5 days and 10 lecture halls
columns = [f'lh{i}' for i in range(15)]
data = [[None for _ in range(15)] for _ in range(5)]
timetable = pd.DataFrame(data, columns=columns)
temp_lh_list = []
# GA parameters
POPULATION_SIZE = 100
GENES = list(range(len(course_metadata)))  # Courses are represented by indices in course_metadata
MAX_GENERATIONS = 200
ANNEALING_THRESHOLD = 50  # Apply simulated annealing if no improvement in 50 generations
day_ctr = 0
# Individual class
class Individual:
    def __init__(self, chromosome):
        self.chromosome = chromosome
        self.fitness = self.calculate_fitness()

    @classmethod
    def create_gnome(cls):
        return [random.choice(GENES) for _ in range(8)]

    @classmethod
    def mutate_gene(cls):
        return random.choice(GENES)

    def mate(self, partner):
        child_chromosome = []
        for gene1, gene2 in zip(self.chromosome, partner.chromosome):
            prob = random.random()
            if prob < 0.45:
                child_chromosome.append(gene1)
            elif prob < 0.90:
                child_chromosome.append(gene2)
            else:
                child_chromosome.append(self.mutate_gene())
        return Individual(child_chromosome)

    def calculate_fitness(self):
        fitness = 0
        counts = Counter(self.chromosome)
        flattened_templh_list = [item for sublist in temp_lh_list if sublist is not None for item in sublist]
        counts_flattened = Counter(flattened_templh_list)
        # Constraints for credit hours
        for i, course_index in enumerate(self.chromosome):
            if course_metadata.iloc[course_index]['Credit Hours'] <= 0:
                fitness += 1
            if counts[course_index] > course_metadata.iloc[course_index]['Credit Hours']:
                fitness += counts[course_index] - course_metadata.iloc[course_index]['Credit Hours']
            # avoiding overlaps
            for lh_slots in temp_lh_list:
                if lh_slots[i] == course_index:
                    fitness+=1
                # avoiding teacher overlaps
                if course_metadata.iloc[lh_slots[i]]["Teacher ID"] == course_metadata.iloc[course_index]["Teacher ID"] :
                    fitness+=1
                # avoiding same section clash
                if course_metadata.iloc[lh_slots[i]]["Semester"] == course_metadata.iloc[course_index]["Semester"] :
                    if course_metadata.iloc[lh_slots[i]]["Section"] == course_metadata.iloc[course_index]["Section"] :
                        fitness+=1
            if counts_flattened[course_index] >= 2:
                fitness += counts_flattened[course_index]
            # Wednesday slot 11 30 to 12 30 should be reserved for seminar
            if day_ctr == 2:
                if i == 4 and course_index == 102:
                    fitness -= 3

        return fitness

def tournament_selection(population):
    tournament = random.sample(population, k=5)
    return min(tournament, key=lambda ind: ind.fitness)

def reduce_credit_hours(course_list):
    for course in course_list:
        course_metadata.loc[course, 'Credit Hours'] -= 1

def simulated_annealing(individual, temperature=1.0, cooling_rate=0.95):
    current = individual
    best = individual
    while temperature > 0.01:
        new_individual = Individual([gene if random.random() > 0.2 else random.choice(GENES) for gene in current.chromosome])
        if new_individual.fitness < best.fitness:
            best = new_individual
        elif random.random() < np.exp((current.fitness - new_individual.fitness) / temperature):
            current = new_individual
        temperature *= cooling_rate
    return best

def initialize_population():
    # Greedy initialization for a better start
    population = []
    for _ in range(POPULATION_SIZE // 2):
        chromosome = [random.choice(GENES) for _ in range(8)]
        population.append(Individual(chromosome))
    # Adding some completely random individuals
    for _ in range(POPULATION_SIZE // 2):
        chromosome = Individual.create_gnome()
        population.append(Individual(chromosome))
    return population

def main():
    ctr = 0
    global timetable
    global temp_lh_list
    global day_ctr
    for day in timetable.index:
        print("Processing day:", day)
        temp_lh_list = []
        for lh in timetable.columns:
            generation = 1
            found = False
            population = initialize_population()
            no_improvement_count = 0
            best_fitness = float('inf')
            
            while not found and generation < MAX_GENERATIONS:
                population.sort(key=lambda x: x.fitness)

                # Check for best individual
                if population[0].fitness <= 0:
                    found = True
                    timetable.loc[day, lh] = population[0].chromosome
                    reduce_credit_hours(population[0].chromosome)
                    temp_lh_list.append(population[0].chromosome)
                    ctr+=1
                    print(ctr)
                    break

                # If no improvement in fitness, apply simulated annealing
                if population[0].fitness < best_fitness:
                    best_fitness = population[0].fitness
                    no_improvement_count = 0
                else:
                    no_improvement_count += 1
                    if no_improvement_count >= ANNEALING_THRESHOLD:
                        population[0] = simulated_annealing(population[0])
                        no_improvement_count = 0

                # New generation using elitism and mating
                new_generation = []
                new_generation.extend(population[:int(0.15 * POPULATION_SIZE)])  # Elitism
                for _ in range(int(0.7 * POPULATION_SIZE)):
                    parent1 = tournament_selection(population)
                    parent2 = tournament_selection(population)
                    child = parent1.mate(parent2)
                    new_generation.append(child)
                for _ in range(int(0.15 * POPULATION_SIZE)):
                    new_generation.append(Individual(Individual.create_gnome()))  # Random mutation
                
                population = new_generation
                print(day,lh, generation)
                generation += 1
        day_ctr += 1

if __name__ == "__main__":
    main()
    print(timetable)


Processing day: 0
1
2
3
4
5
0 lh5 1
6
0 lh6 1
7
0 lh7 1
0 lh7 2
0 lh7 3
8
0 lh8 1
0 lh8 2
0 lh8 3
9
0 lh9 1
0 lh9 2
0 lh9 3
0 lh9 4
0 lh9 5
10
0 lh10 1
0 lh10 2
0 lh10 3
0 lh10 4
0 lh10 5
0 lh10 6
11
0 lh11 1
0 lh11 2
0 lh11 3
0 lh11 4
0 lh11 5
0 lh11 6
12
0 lh12 1
0 lh12 2
0 lh12 3
0 lh12 4
0 lh12 5
0 lh12 6
0 lh12 7
13
0 lh13 1
0 lh13 2
0 lh13 3
0 lh13 4
0 lh13 5
0 lh13 6
0 lh13 7
0 lh13 8
0 lh13 9
0 lh13 10
0 lh13 11
0 lh13 12
0 lh13 13
0 lh13 14
0 lh13 15
0 lh13 16
0 lh13 17
0 lh13 18
0 lh13 19
0 lh13 20
0 lh13 21
0 lh13 22
0 lh13 23
0 lh13 24
0 lh13 25
0 lh13 26
0 lh13 27
0 lh13 28
0 lh13 29
0 lh13 30
0 lh13 31
0 lh13 32
0 lh13 33
0 lh13 34
0 lh13 35
0 lh13 36
0 lh13 37
0 lh13 38
0 lh13 39
0 lh13 40
0 lh13 41
0 lh13 42
0 lh13 43
0 lh13 44
0 lh13 45
0 lh13 46
0 lh13 47
14
0 lh14 1
0 lh14 2
0 lh14 3
0 lh14 4
0 lh14 5
0 lh14 6
0 lh14 7
0 lh14 8
0 lh14 9
0 lh14 10
0 lh14 11
0 lh14 12
0 lh14 13
0 lh14 14
0 lh14 15
0 lh14 16
0 lh14 17
0 lh14 18
0 lh14 19
0 lh14 20
0 lh14 21
0 lh14 22
0 

In [107]:
timetable

,lh0,lh1,lh2,lh3,lh4,lh5,lh6,lh7,lh8,lh9,lh10,lh11,lh12,lh13,lh14
0,"[17, 59, 13, 68, 59, 9, 83, 93]","[80, 3, 92, 32, 27, 5, 31, 85]","[28, 23, 26, 4, 45, 56, 90, 83]","[9, 100, 34, 42, 93, 43, 27, 19]","[70, 11, 21, 49, 98, 54, 18, 49]","[96, 45, 67, 48, 43, 55, 15, 2]","[82, 13, 98, 33, 74, 91, 53, 34]","[88, 78, 42, 52, 46, 1, 61, 77]","[48, 85, 44, 10, 86, 11, 3, 18]","[0, 55, 71, 64, 71, 67, 67, 74]","[69, 102, 97, 88, 8, 23, 99, 64]","[102, 0, 54, 102, 87, 94, 12, 65]","[5, 80, 6, 61, 79, 17, 41, 69]","[66, 8, 41, 62, 65, 73, 72, 70]",None
1,"[6, 24, 93, 21, 26, 54, 92, 91]","[23, 74, 6, 65, 66, 99, 38, 86]","[66, 56, 19, 46, 30, 28, 57, 28]","[86, 80, 12, 44, 97, 78, 51, 97]","[44, 61, 32, 16, 84, 57, 9, 88]","[10, 7, 102, 60, 62, 51, 2, 27]","[12, 59, 33, 14, 72, 32, 73, 50]","[90, 76, 60, 47, 2, 36, 26, 42]","[84, 45, 8, 69, 55, 79, 64, 15]","[34, 50, 21, 71, 24, 7, 89, 73]","[62, 11, 17, 33, 18, 25, 102, 53]","[68, 48, 63, 31, 14, 16, 72, 81]","[75, 1, 70, 19, 79, 68, 63, 5]",None,None
2,"[13, 81, 91, 90, 102, 6, 0, 1]","[49, 102, 87, 51, 102, 36, 43, 29]","[50, 87, 7, 39, 101, 47, 35, 39]","[30, 95, 36, 76, 47, 31, 89, 24]","[52, 10, 16, 101, 41, 81, 22, 57]","[35, 58, 40, 35, 3, 77, 63, 14]","[53, 4, 52, 25, 76, 38, 95, 25]",None,None,None,None,None,None,None,None
3,"[30, 99, 58, 101, 83, 92, 40, 60]","[37, 37, 4, 29, 89, 29, 100, 96]","[75, 15, 75, 94, 77, 102, 102, 102]","[95, 96, 85, 39, 20, 94, 82, 84]",None,None,None,None,None,None,None,None,None,None,None
4,"[58, 100, 37, 82, 98, 20, 38, 102]","[102, 20, 22, 102, 40, 102, 56, 22]",None,None,None,None,None,None,None,None,None,None,None,None,None


In [108]:
# 102 id malfunctionaing